In [57]:
from binance.client import Client
from binance.enums import *
from time import time
import pickle as pickle
from datetime import datetime
import pandas as pd

def get_client():
    fn = '../../key/binance-key.pickle'
    # fn = '/home/era/key/binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])


client = get_client()


def get_unix_timestamp(date_string):
    """
    Converts the input date string to Unix timestamp.

    Parameters:
        date_string (str): Input date string in the format "dd/mm/yyyy hh:mm:ss".

    Returns:
        int: Unix timestamp of the given date.
    """
    try:
        date_obj = datetime.strptime(date_string, "%d/%m/%Y %H:%M:%S")
        timestamp = int(date_obj.timestamp())
        return timestamp
    except ValueError:
        print("Invalid date format. Please use the format 'dd/mm/yyyy hh:mm:ss'.")
        return None
def ts_to_local_time(ts):
    UTC_OFFSET = 14400
    lt = datetime.utcfromtimestamp(ts+UTC_OFFSET).strftime('%d-%m-%Y %H:%M:%S')
    return lt

def calc_compounded_pnl(pnls):
    try:
        s=100.0
        for i in pnls:
            s = (1+(i/100))*s
        return round(s - 100.0,1)
    except:
        return 0

def get_historical_data(coin_pair, interval, start_timestamp, end_timestamp): 
    data = []
    tot = (end_timestamp - start_timestamp)/(900*500)
    cntr = 0
    for current_sts in range(start_timestamp, end_timestamp+1, 900*500):
        next_ets = current_sts + 900*500 if (current_sts + 900*500) < end_timestamp else end_timestamp
        print(current_sts, next_ets, f'100% completed') if next_ets == end_timestamp else print(current_sts, next_ets, f'{round(cntr*100/tot, 1)}% completed')
        cntr += 1
        # Futures market
        klines = client.futures_historical_klines(coin_pair, interval, current_sts*1000, next_ets*1000, limit=500)
        # Spot market
        # klines = client.get_historical_klines(coin_pair, interval, current_sts*1000, next_ets*1000, limit=500)
        
        for kline in klines:
            timestamp = kline[0]/1000
            open_price = float(kline[1])
            high_price = float(kline[2])
            low_price = float(kline[3])
            close_price = float(kline[4])
            volume = float(kline[5])

            data.append([timestamp, open_price, high_price, low_price, close_price, volume])

    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    # df.to_csv(f'../../data/{coin_pair}-{interval}.csv', index=False)
    return df

# coin_pair = 'ETHUSDT'
# interval = '15m'

# # Thursday, August 17, 2017 4:00:00 AM
# start_timestamp = get_unix_timestamp('1/1/2016 00:00:00')
# end_timestamp = int(time())
# # end_timestamp = get_unix_timestamp('1/1/2022 00:00:00')


# get_historical_data(start_timestamp, end_timestamp)


In [58]:
def sim_trade(coin_pair, sig, start_timestamp, end_timestamp):
    hd = get_historical_data(coin_pair, '1m', start_timestamp, end_timestamp)
    tm = list(hd['time'])
    entyr_price = float(list(hd['open'])[0])
    high = list(hd['high'])
    low = list(hd['low'])
    close = list(hd['close'])
    tp = 3.0
    pnl = 0.0
    for i in range(len(tm)):
        if sig == 'short':
            pnl = round((entyr_price*100/low[i])-100,2)
        if sig == 'long':
            pnl = round((high[i]*100/entyr_price)-100,2)
        if pnl >= tp:
            return pnl, tm[i]
    return pnl, int(tm[i])


signal_log_df = pd.read_csv('../../data/signal_log.csv')
signal = list(signal_log_df['Signal'])
confidence_Score = list(signal_log_df['Confidence_Score'])
coin = list(signal_log_df['Coin'])
log_Time = list(signal_log_df['Log_Time'])
logTS = list(signal_log_df['LogTS'])


In [59]:
last_ts = 0
dta = []
for idx, start_timestamp in enumerate(logTS):
    if (start_timestamp >= last_ts) and (confidence_Score[idx] >= 0.5):
        sig = signal[idx]
        end_timestamp = start_timestamp + 10800
        coin_pair = coin[idx]
        pnl, last_ts = sim_trade(coin_pair, sig, start_timestamp, end_timestamp)
        entry_time = ts_to_local_time(start_timestamp)
        exit_time = ts_to_local_time(last_ts)
        dta.append({'coin_pair': coin_pair, 'pnl': pnl, 'entry_time': entry_time, 'exit_time': exit_time, 'entry_ts': start_timestamp, 'exit_ts': last_ts})


1692937878 1692948678 100% completed
1692948680 1692959480 100% completed
1692959479 1692970279 100% completed
1692970280 1692981080 100% completed
1692981082 1692991882 100% completed
1692992779 1693003579 100% completed
1693003579 1693014379 100% completed
1693014380 1693025180 100% completed
1693025179 1693035979 100% completed
1693035978 1693046778 100% completed
1693046781 1693057581 100% completed


In [60]:
exp_df = pd.DataFrame(dta)
pnls = list(exp_df['pnl'])
c_pnl = calc_compounded_pnl(pnls)
exp_df['compounded_pnl'] = [c_pnl]+['']*((len(pnls))-1)
exp_df.to_csv(f'../../data/tst.csv', index=False)
